In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [ ]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [ ]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

**Policy Evaluation**

In [ ]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            #print(f'Converged at {i}-th iteration.')
            break

    return v_values

**Policy Extraction**

In [ ]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

**Value Iteration**

In [ ]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Value Iteration converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
def VI(env):
  print ("VALUATION ITERATION")
  start_time = time.time()
  v = value_iteration(env, max_iters=500, gamma=0.9)
  p = policy_extraction(env, v, gamma=0.9)
  end_time = time.time()
  print (f"+ Time: {end_time - start_time}" )
  print("+ Values:\n", v)
  print("+ Poilicy: ", p)
  print ("+ Result: ")
  play_multiple_times(env, p, 1000)

#**Policy Iteration**

In [ ]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)
    v_values = np.zeros(env.observation_space.n)
    for i in range(max_iters):
        # Evaluation Policy
        pre_policy = np.copy(policy)
        v_values = policy_evaluation(env, policy, max_iters, gamma)

        # Improvement Policy
        policy = policy_extraction(env, v_values, gamma)

        # check convergence
        if np.all(policy == pre_policy):
            print(f'Policy Iteration converged at {i}-th iteration')
            break
    return policy, v_values

In [ ]:
def PI(env):
  print ("POLICY ITERATION")
  start_time = time.time()
  p,v = policy_iteration(env, max_iters=500, gamma=0.9)
  end_time = time.time()
  print (f"+ Time: {end_time - start_time}" )
  print("+ Values:\n", v)
  print("+ Poilicy: ", p)
  print ("+ Result: ")
  play_multiple_times(env, p, 1000)

#**FrozenLake-v1**

In [ ]:
env1 = gym.make('FrozenLake-v1', render_mode="ansi")

In [ ]:
VI(env1)

VALUATION ITERATION
Value Iteration converged at 79-th iteration.
+ Time: 0.0739133358001709
+ Values:
 [0.06888615 0.06141054 0.07440682 0.05580409 0.09185022 0.
 0.11220663 0.         0.14543286 0.2474946  0.29961593 0.
 0.         0.3799342  0.63901926 0.        ]
+ Poilicy:  [0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
+ Result: 
Number of successes: 804/1000
Average number of steps: 43.385572139303484


In [ ]:
PI(env1)

POLICY ITERATION
Policy Iteration converged at 5-th iteration
+ Time: 0.10219907760620117
+ Values:
 [0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]
+ Poilicy:  [0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
+ Result: 
Number of successes: 777/1000
Average number of steps: 45.13256113256113


#**FrozenLake8x8-v1**

In [ ]:
env2 = gym.make('FrozenLake8x8-v1', render_mode="ansi")

In [ ]:
VI(env2)

VALUATION ITERATION
Value Iteration converged at 117-th iteration.
+ Time: 0.6902282238006592
+ Values:
 [0.00641104 0.00854808 0.01230044 0.01778942 0.02508214 0.03247089
 0.03957134 0.04297844 0.00602405 0.00764512 0.01091162 0.01642654
 0.02605411 0.03619409 0.0493547  0.05730461 0.00509024 0.0058532
 0.00677534 0.         0.02557084 0.03882139 0.06763973 0.08435607
 0.0042256  0.00476954 0.00581968 0.0078541  0.02036065 0.
 0.09175501 0.12919111 0.00318093 0.00319659 0.00270488 0.
 0.0344439  0.06195145 0.10901921 0.20969093 0.00186915 0.
 0.         0.01085079 0.03250092 0.06304172 0.         0.36008773
 0.00118046 0.         0.00137717 0.00366839 0.         0.11568671
 0.         0.63051379 0.00088531 0.00077462 0.00092218 0.
 0.13824885 0.32258065 0.61443932 0.        ]
+ Poilicy:  [3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0]
+ Result: 
Number of successes: 752/1000
Average number of steps: 75.

In [ ]:
PI(env2)

POLICY ITERATION
Policy Iteration converged at 9-th iteration
+ Time: 1.0815684795379639
+ Values:
 [0.00641104 0.00854808 0.01230044 0.01778942 0.02508214 0.03247089
 0.03957134 0.04297844 0.00602405 0.00764512 0.01091162 0.01642654
 0.02605411 0.03619409 0.0493547  0.05730461 0.00509024 0.0058532
 0.00677534 0.         0.02557084 0.03882139 0.06763973 0.08435607
 0.0042256  0.00476954 0.00581968 0.0078541  0.02036065 0.
 0.09175501 0.12919111 0.00318093 0.00319659 0.00270488 0.
 0.0344439  0.06195145 0.10901921 0.20969093 0.00186915 0.
 0.         0.01085079 0.03250092 0.06304172 0.         0.36008773
 0.00118046 0.         0.00137717 0.00366839 0.         0.11568671
 0.         0.63051379 0.0008853  0.00077462 0.00092218 0.
 0.13824885 0.32258065 0.61443932 0.        ]
+ Poilicy:  [3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0]
+ Result: 
Number of successes: 770/1000
Average number of steps: 74.50909

#**Taxi-v3**

In [ ]:
env3 = gym.make('Taxi-v3', render_mode="ansi")

In [ ]:
VI(env3)

VALUATION ITERATION
Value Iteration converged at 116-th iteration.
+ Time: 5.384363889694214
+ Values:
 [ 89.47323891  32.81971401  55.26423891  37.57755845   8.43222921
  32.81971401   8.43222921  15.28447953  32.81971401  18.09386122
  55.26423891  21.2154998   12.75594298  18.09386122  12.75594298
  37.57755845 100.52591945  37.57755845  62.51591945  42.86394891
  79.52591945  28.53774704  48.73781945  32.81971401  10.48035311
  37.57755845  10.48035311  18.09386122  28.53774704  15.28447953
  48.73781945  18.09386122  15.28447953  21.2154998   15.28447953
  42.86394891  89.47323891  42.86394891  55.26423891  48.73781945
  42.86394891  12.75594298  24.68388374  15.28447953  24.68388374
  70.57323891  24.68388374  37.57755845  24.68388374  12.75594298
  42.86394891  15.28447953  18.09386122  24.68388374  18.09386122
  48.73781945  48.73781945  79.52591945  48.73781945  55.26423891
  37.57755845  10.48035311  21.2154998   12.75594298  28.53774704
  79.52591945  28.53774704  42.8639489

In [ ]:
PI(env3)

POLICY ITERATION
Policy Iteration converged at 16-th iteration
+ Time: 14.158045053482056
+ Values:
 [ 89.47323891  32.81971401  55.26423891  37.57755845   8.43222921
  32.81971401   8.43222921  15.28447953  32.81971401  18.09386122
  55.26423891  21.2154998   12.75594298  18.09386122  12.75594298
  37.57755845 100.52591945  37.57755845  62.51591945  42.86394891
  79.52591945  28.53774704  48.73781945  32.81971401  10.48035311
  37.57755845  10.48035311  18.09386122  28.53774704  15.28447953
  48.73781945  18.09386122  15.28447953  21.2154998   15.28447953
  42.86394891  89.47323891  42.86394891  55.26423891  48.73781945
  42.86394891  12.75594298  24.68388374  15.28447953  24.68388374
  70.57323891  24.68388374  37.57755845  24.68388374  12.75594298
  42.86394891  15.28447953  18.09386122  24.68388374  18.09386122
  48.73781945  48.73781945  79.52591945  48.73781945  55.26423891
  37.57755845  10.48035311  21.2154998   12.75594298  28.53774704
  79.52591945  28.53774704  42.86394891  

#**Nhận xét**

<html>
<body>
<h1><b>Bảng thống kê</b></h1>
<table>
<tr>
<th rowspan = 2>Environment</th>
<th colspan = 2>Valuation Iteration</th>
<th colspan = 2>Policy Iteration</th>
</tr>
<tr>
<th scope = "col">Converged at</th>
<th scope = "col">Time (s)</th>
<th scope = "col">Converged at</th>
<th scope = "col">Time (s)</th>
</tr>
<tr>
<th scope = "row">FrozenLake-v1</th>
<td align="center">79-th iteration</td>
<td align="center">0.0739133358001709</td>
<td align="center">5-th iteration</td>
<td align="center">0.10219907760620117</td>
</tr>
<tr>
<th scope = "row">FrozenLake8x8-v1</th>
<td align="center">117-th iteration</td>
<td align="center">0.6902282238006592</td>
<td align="center">9-th iteration</td>
<td align="center">1.0815684795379639</td>
</tr>
<tr>
<th scope = "row">Taxi-v3</th>
<td align="center">116-th iteration</td>
<td align="center">5.384363889694214</td>
<td align="center">16-th iteration</td>
<td align="center">14.158045053482056</td>
</tr>
</table>
<h1><b>Nhận xét </b></h1>
</body>
</html>

Ở cả 3 toy games:
- Valuation Iteration và Policy Iteration cho ra **kết quả** (optional policy, optinal state-value function) **gần giống nhau**.

- Valuation Iteration cho ra kết quả nhanh hơn. **Do ở mỗi vòng lặp**:
  + Valuation Iteration chỉ cập nhật state-value function, sau khi thuật toán hội tụ (tìm được *optional state-value function*) mới đi tìm *optional policy* bằng cách sử dụng hàm policy_extraction.
  + Policy Iteration phải làm 2 việc là Policy Evaluation và Policy Improvement nên mất nhiều thời gian hơn so với Valuation Iteration đối với mỗi vòng lặp.

- Tuy nhiên, Policy Iteration chạy ít vòng lặp hơn để hội tụ.

